# Table Page

> The page of the app that generates the data table view.

In [ ]:
#| default_exp pages/table_page

In [ ]:
#| export
# TODO minimise these imports a bit where possible
from nbdev.showdoc import *
# TODO work out how to get around below hack
try:
    from SolomonIslandsDataMap.dash_components import gen_census_grid, gen_pop_grid, gen_dd
    from SolomonIslandsDataMap.app_data import mytitle, data_grid, download_button, fake_slider
    from SolomonIslandsDataMap.load_data import SolomonGeo
except: 
    from dash_components import gen_census_grid, gen_pop_grid, gen_dd
    from app_data import mytitle, data_grid, download_button, fake_slider
    from load_data import SolomonGeo
from fastcore.test import *
from dash import Dash, dcc, callback, Output, Input, State, html, Patch, ctx, register_page, callback_context 
import dash_bootstrap_components as dbc    
import dash_ag_grid as dag

In [ ]:
#| export
try:
    register_page(__name__, 
                        path='/datatable',
                        title='Data Table',
                        name='Data Table')
except:
    pass

Setup some of  the key data contiaining objects

In [ ]:
#| export
init_init = dcc.Store(id="initial-initial", data='table')
def layout():
    return dbc.Row([
                dbc.Col(mytitle, width = 8),
                dbc.Col(download_button, width = {"size": 2})
                      ]), data_grid, init_init, fake_slider


# Dash Callbacks
Create interactivity in the data table

### Create a callback that always triggers the map data and bargaph to update on map load
I did this as the array of callbacks are quite complex and others are triggering the maps initial load

#| export
# TODO this defintiely seems hacky, must be a better way

@callback(
    Output('initial-load', 'data'),
    Input(init_init, 'data')
)
def initial_load_table(fakeData:{}, # A fake variable, ignore
                    ) -> None: 
    """This just triggers initial load"""
    return None
        

show_doc(initial_load_table)  

## Update the data displayed in the table

In [ ]:
#| export
@callback(
    Output('dataGridContainer', 'children'),
    Input("segmented_geo", 'value'),
    Input("segmented_type", 'value'),
    Input('measureDropdown', 'value'),
    Input('varDropdown', 'value'),
    Input('measureDropdownPop', 'value'),
    Input('varDropdownPop', 'value'),
    Input('grid-rows', 'value'),
    Input("dataset_type", "value"), 
    State('geo_df', 'data'),
    allow_duplicate=True,
)
def update_grid(geo_input:str, # User input from the geography dropdownk
                    data_type:str, # User input of type of data
                    measure:str, # A string contiaining the census variable and measure split by ':'
                    variable:str, # The state of the variable dropdown
                    measurePop:str, # A string contiaining the population measure
                    variablePop:str, # The variable to display for population dataset
                    grid_rows:int, # The number of rows to display
                    dataset:str, # The dataset to load
                    dict_sol:dict, # The dataset in dictionary form
              )->(dag.AgGrid, str): # Returns a graph object figure after being updated and the dynamic title
    '''
    Updates the focus census variable or geography dispalayed on the map
    '''
    # TODO add proportion functionality
    # TODO add add an option for first call to be a default rebuild
    # TODO decide wether to implment patch later
    patched_figure = Patch()
    button_clicked = ctx.triggered_id

    sol_geo = SolomonGeo.gen_stored(dict_sol) # reload the data

    if button_clicked == 'locDropdown':
        # Update disaplayed geography 
        # TODO in future update row highlighting
        return patched_figure
    elif dataset == 'Census' and\
    button_clicked in ["segmented_type", "segmented_geo",  'varDropdown',  'measureDropdown', 'dataset_type', None]:
        patched_figure = gen_census_grid(sol_geo, geo_input,variable, measure, 
            type_filter = data_type, grid_rows=grid_rows)
    elif dataset == 'Population Projections' and\
    button_clicked in ["segmented_type", "segmented_geo", 'varDropdownPop',  'measureDropdownPop', 'dataset_type', None]:
        patched_figure = gen_pop_grid(sol_geo, sol_geo.pop_years, variablePop, measurePop, geo_filter = geo_input,
            type_filter = data_type, grid_rows=grid_rows)
    elif button_clicked == 'grid-rows':
        # In this case, update using patch
        patched_figure.dashGridOptions['paginationPageSize'] = grid_rows

    return patched_figure

In [ ]:
show_doc(update_grid)

---

### update_grid

>      update_grid (geo_input:str, data_type:str, measure:str, variable:str,
>                   measurePop:str, variablePop:str, grid_rows:int, dataset:str,
>                   dict_sol:dict)

Updates the focus census variable or geography dispalayed on the map

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| geo_input | str | User input from the geography dropdownk |
| data_type | str | User input of type of data |
| measure | str | A string contiaining the census variable and measure split by ':' |
| variable | str | The state of the variable dropdown |
| measurePop | str | A string contiaining the population measure |
| variablePop | str | The variable to display for population dataset |
| grid_rows | int | The number of rows to display |
| dataset | str | The dataset to load |
| dict_sol | dict | The dataset in dictionary form |
| **Returns** | **(<class 'dash_ag_grid.AgGrid.AgGrid'>, <class 'str'>)** | **Returns a graph object figure after being updated and the dynamic title** |

## Update the Grid row count
Needs to both:
- update max input based on the geography selected
- update the dash_grid size based on the selected row count

In [ ]:
#| export
@callback(
    Output("grid-rows", "value"),
    Output("grid-rows", "max"),
    Input("segmented_geo", 'value'),
    Input("dataset_type", 'value'),
    State("geo_df", 'data'),
    suppress_callback_exceptions = True,
)
def update_page_rows(geo_input:str, # User input from the geography dropdown
                    dataset:str, # Currently selected dataset 
                    dict_sol:dict, # The dataset in dictionary form
                    ) -> (int, int): # New value and max size of the grid selection
    ''' Updates the page size and max of the input selection based on updated geography'''
    sol_geo = SolomonGeo.gen_stored(dict_sol) # reload the data
    max_rows = len(sol_geo.locations[geo_input])
    if dataset == 'Population Projections': max_rows = max_rows * len(sol_geo.pop_years)
    return 10, max_rows

In [ ]:
show_doc(update_page_rows)

---

### update_page_rows

>      update_page_rows (geo_input:str, dataset:str, dict_sol:dict)

Updates the page size and max of the input selection based on updated geography

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| geo_input | str | User input from the geography dropdown |
| dataset | str | Currently selected dataset |
| dict_sol | dict | The dataset in dictionary form |
| **Returns** | **(<class 'int'>, <class 'int'>)** | **New value and max size of the grid selection** |

In [ ]:
#| export
@callback(
    Output("dash-grid", "dashGridOptions"),
    Input("grid-rows", "value"),
    State("dash-grid", "dashGridOptions"),
    suppress_callback_exceptions = True,
)
def update_page_size(page_size:int, # The input page size
                    grid_options:dict, # The current grid page size option
                    ) -> dict: # The updated grid page size option
    ''' Updates the page size of the grid based on input'''
    page_size = 1 if page_size is None else page_size
    grid_options["paginationPageSize"] = page_size
    return grid_options

In [ ]:
show_doc(update_page_size)

---

### update_page_size

>      update_page_size (page_size:int, grid_options:dict)

Updates the page size of the grid based on input

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| page_size | int | The input page size |
| grid_options | dict | The current grid page size option |
| **Returns** | **dict** | **The updated grid page size option** |

## Enable downloading the current csv
On click of download button, return the current csv in the dash grid

In [ ]:
#| export
@callback(
    Output("dash-grid", "exportDataAsCsv"),
    [Input('csv-button', "n_clicks")],
)
def export_data_as_csv(n_clicks:int, # Listening for click inputs
                       ) -> bool: # Whether to download csv
    trigger = callback_context.triggered[0]
    if trigger['prop_id'] ==  'csv-button.n_clicks':
        return True
    return False

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
data_grid.children.columnDefs

[{'field': 'Province',
  'headerName': 'Province',
  'filter': True,
  'lockPinned': True,
  'cellClass': 'lock-pinned'},
 {'field': 'Total Households', 'headerName': 'Total Households'}]